# Introduction

Intron annotation files I used can come from two sources: 

1. using leafcutter's leafviz/gtf2leafcutter.pl script
2. using gtftookits `gtftk` methods

See details of extracting annotations from gencode gtf files from either of the
two methods in snakefile (~/cdai/annotations/workflow/Snakefile)


## Conclusion

Leafcutter's `leafviz/gtf2leafcutter.pl` produces non-standard intron coordinates. Even though the produced file is using `.bed`
file extension, the file is not in correct BED format!

- The start position is in correct BED format. 
- The end position is WRONG! The reported end value from file is 1 greater than the correct BED formatted end value!

**use `gtftk`, do not use `gtf2leafcutter.pl`!**


## Details 

see below: ...

In [14]:
#| label: set-up

import os
import pandas as pd

In [16]:
os.getcwd()

'/project2/yangili1/cdai/annotations'

In [10]:
os.chdir('../../')

In [18]:
#| include: false

!ls -lahR hg38

hg38:
total 48M
drwxrwsr-x 4 chaodai pi-yangili1 4.0K May 12 14:50 .
drwxrwsr-x 6 chaodai pi-yangili1 4.0K May 12 17:57 ..
-rw-rw-r-- 1 chaodai pi-yangili1  48M May 12 12:25 gencode.v43.primary_assembly.annotation.gtf.gz
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 12 18:42 use_gtftk
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 12 18:24 use_leafviz

hg38/use_gtftk:
total 11M
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 12 18:42 .
drwxrwsr-x 4 chaodai pi-yangili1 4.0K May 12 14:50 ..
-rw-rw-r-- 1 chaodai pi-yangili1 4.7M May 12 18:15 gencode_v43_basic.exons.bed.gz
-rw-rw-r-- 1 chaodai pi-yangili1 4.4M May 12 18:19 gencode_v43_basic.intron_by_transcript.bed.gz
-rw-rw-r-- 1 chaodai pi-yangili1 1.6M May 12 18:44 gencode_v43_basic.transcript.bed.gz

hg38/use_leafviz:
total 21M
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 12 18:24 .
drwxrwsr-x 4 chaodai pi-yangili1 4.0K May 12 14:50 ..
-rw-rw-r-- 1 chaodai pi-yangili1    0 May 12 18:24 done
-rw-rw-r-- 1 chaodai pi-yangili1 5.2M May 12 18:24 gencode_v43_

## Introns from gtftk

In [20]:
intron_gtftk = pd.read_csv('hg38/use_gtftk/gencode_v43_basic.intron_by_transcript.bed.gz', sep='\t', header=None, 
                           names=['chrom', 'start', 'end', 'name', 'score', 'strand'])

In [21]:
intron_gtftk.iloc[1:2]
intron_gtftk.shape

,chrom,start,end,name,score,strand
1,chr1,12721,13220,intron|ENSG00000290825.1|ENST00000456328.2|DDX...,2,+


(486009, 6)

In [22]:
intron_leaf = pd.read_csv('hg38/use_leafviz/gencode_v43_all_introns.bed.gz', sep='\t', header=None,
                         names=['chrom', 'start', 'end', 'gname', 'gid', 'strand', 'tid', 'score', 'gene_type', 'tags'])

In [23]:
intron_leaf.iloc[1:2,:]
intron_leaf.shape

,chrom,start,end,gname,gid,strand,tid,score,gene_type,tags
1,chrX,100635252,100635558,TSPAN6,ENSG00000000003.15,-,ENST00000373020.9,2,protein_coding,basic|Ensembl_canonical|MANE_Select|appris_pri...


(486009, 10)

In [37]:
intron_gtftk[intron_gtftk['name'].str.contains('ZBTB18')].sort_values(['name', 'start']).reset_index(drop=True)

,chrom,start,end,name,score,strand
0,chr1,244051444,244053787,intron|ENSG00000179456.12|ENST00000358704.4|ZB...,1,+
1,chr1,244048993,244053787,intron|ENSG00000179456.12|ENST00000622512.1|ZB...,1,+


## Introns from leafcutter perl script

In [38]:
intron_leaf[intron_leaf.gname.str.contains('ZBTB18')].sort_values(['tid', 'start']).reset_index(drop=True)

,chrom,start,end,gname,gid,strand,tid,score,gene_type,tags
0,chr1,244051444,244053788,ZBTB18,ENSG00000179456.12,+,ENST00000358704.4,1,protein_coding,basic|Ensembl_canonical|MANE_Select|appris_pri...
1,chr1,244048993,244053788,ZBTB18,ENSG00000179456.12,+,ENST00000622512.1,1,protein_coding,basic|appris_alternative_1


## Comparison notes: 

1. Notice for each gene, the number of introns from either method are the same. 
2. gtftk produced introns follow correct BED format, however, leafcutter produced results do not follow correct BEDformat. 
    - Specifically, leafcutter produced introns have the **same start coordinate, but an increase by 1 end coordinate**.

## Check gtftk produced intron junctions, along with transcript and exon coordiantes

1. example for ZBTB18 shows the intron for transcript 704.4 is from ...444(close) to ...787(open), while first exons ends at ...444(open) and the second
exon starts from ...787 (close). 
2. similarly the second intron is also correct, considering all transcript, exons, and introns are strictly BED format.

In [43]:
# transcripts
!zcat hg38/use_gtftk/gencode_v43_basic.transcript.bed.gz | awk '$4 ~ /ZBTB18/' | sort -k4 -k2

chr1	244051282	244057476	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244048938	244057472	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+


In [44]:
# exons
!zcat hg38/use_gtftk/gencode_v43_basic.exons.bed.gz | awk '$4 ~ /ZBTB18/' | sort -k4 -k2

chr1	244051282	244051444	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244053787	244057476	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244048938	244048993	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+
chr1	244053787	244057472	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+


In [46]:
# introns
intron_gtftk[intron_gtftk['name'].str.contains('ZBTB18')].sort_values(['name', 'start']).reset_index(drop=True)

,chrom,start,end,name,score,strand
0,chr1,244051444,244053787,intron|ENSG00000179456.12|ENST00000358704.4|ZB...,1,+
1,chr1,244048993,244053787,intron|ENSG00000179456.12|ENST00000622512.1|ZB...,1,+


## Check leafcutter produced intron junctions, along with transcript and exon coordiantes

1. exons from leafcutter are 1 based both close format (like VCF or gtf)
2. introns have non-standard coordinates! (**start is 0 based close, but end is non-standard: end appears to be BED-formatted-end + 1**)

In [47]:
# transcripts
# using the same transcript file produced by gtftk
!zcat hg38/use_gtftk/gencode_v43_basic.transcript.bed.gz | awk '$4 ~ /ZBTB18/' | sort -k4 -k2

chr1	244051282	244057476	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244048938	244057472	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+


In [51]:
# exons
!zcat hg38/use_leafviz/gencode_v43_all_exons.txt.gz |  awk '$5 ~ /ZBTB18/' 

chr1	244048939	244048993	+	ZBTB18
chr1	244053788	244057472	+	ZBTB18
chr1	244051283	244051444	+	ZBTB18
chr1	244053788	244057476	+	ZBTB18


In [53]:
# introns
!zcat hg38/use_leafviz/gencode_v43_all_introns.bed.gz | awk '$4 ~ /ZBTB18/'

chr1	244051444	244053788	ZBTB18	ENSG00000179456.12	+	ENST00000358704.4	1	protein_coding	basic|Ensembl_canonical|MANE_Select|appris_principal_4|CCDS
chr1	244048993	244053788	ZBTB18	ENSG00000179456.12	+	ENST00000622512.1	1	protein_coding	basic|appris_alternative_1
